# Generative Adversarial Network

## Useful Links

https://developers.google.com/machine-learning/gan/ <br>
https://lilianweng.github.io/lil-log/2017/08/20/from-GAN-to-WGAN.html#generative-adversarial-network-gan <br>
https://github.com/llSourcell/Pokemon_GAN/blob/master/Generative%20Adversarial%20Networks.ipynb

<div><img src="img/genvsdisc.jpg" width=50%></div>

### Generative Model

- Aim to model how the data is generated. The model tries to captue the joint probability distribution $P(X,Y)$ or $P(X)$ when there are no labels provided.  

### Discriminative Model 

- Aim to model the mapping between input $X$ to the labels $Y$. In terms of probability the model tries capturing $P(Y|X)$

# Methods in ML

- **Generative Methods**
    - model class-conditional probability distributions functions and prior probabilites
    - random sampling generates synthetic data points -> "generative" property
    - example models: Gaussians, Naive Bayes , Mixture of Gaussians, Hidden Markov Models (HMM), Markov Random Fields, Bayesian Networks
    
- **Discriminative Methods** 
    - directly estimate posterior distribution
    - does not try to model underlying probability distribution
    - highly based on training data quality -> mapping input to output
    - example models: Logistic regression, SVM, Neural Networks, Nearest neighbor

- **Discriminative Learning**

    Goal is learning $p(y|x)$. Probability function for the label $y$ given the input $x$.
    
- **Generative Learning**

    Model $p(y)$, $p(x|y)$ first, then derive posterior 
    $p(x|y)=\frac{p(y|x)p(y)}{p(x)}$<br>
    Goal here is modeling the probabilty function for the labels $y$ and the probability for our input $x$ given our label $y$.
    We also model $p(x,y) = p(y)p(x|y)$ through this giving us the joint probability distribution for input $x$ and label $y$ occuring at the same time.
    This represents the underlying data structure.
    
 

Simply illustrating the differnt modeling approaches yields:
- the discriminative approach models the boundary between the different data points based upon the labels/classes
- the generative approach models the data distribution depending on the data points and their label combined.
<div><img src="img/gen_vs_disc.png" width=40%></div>

# Structure of a Generative Adversarial Network (Goodfellow et al. 2014)
<div><img src="img/GANConcept.png" width=50%></div>

The **Generator** draws some random parameters/noise denoted as $z$ from a source of randomness e.g. normal distribution and applies a function $f$ onto to it such that $\hat x = f(z)$ where $ \hat x$ is the output of the generator network. <br>
The **Discriminator** is a binary classifier with the whole purpose being to differentiate between the real input $x$ and the generator input $\hat x$.<br>
The **Loss function** / Objective function can be stated respectively, for the generator we can state $\min_G{p(y=\text{fake}|\hat x)}$ and for the discriminator we want to $\max_D{p (y=\text{fake}|\hat x)}$ and $\max_D{p (y=\text{true}|x)}$.<br>
Seen intuitively the generator is trying to fool the discriminator with its output $\hat x$ to believe that it is a true data point while the discriminator is trying to maximize the accuracy by which it can determine if the input comes from our true data points $x$ or is a fake $\hat x $ by the generator. We can formulate a combined value function using the log-likelihood as follows
$$\begin{aligned}
\min_G \max_D L(D, G) 
& = \mathbb{E}_{x \sim p_{r}(x)} [\log D(x)] + \mathbb{E}_{z \sim p_z(z)} [\log(1 - D(G(z)))] \\
& = \mathbb{E}_{x \sim p_{r}(x)} [\log D(x)] + \mathbb{E}_{x \sim p_g(x)} [\log(1 - D(x)]
\end{aligned}
$$

There are really only 5 components to think about:
- $X:$ The original, genuine data set described by $p_r$
- $I:$ The random noise that goes into the generator as a source of entropy described by $p_z$
- $G:$ The generator which tries to copy/mimic the original data set which output can be described with $p_g$
- $D:$ The discriminator which tries to tell apart $G$’s output from $X$
- Backpropagation / Learning Process

**Training** <br/>
*One training step consists of:*
- Sample a mini batch of **m** noise vectors {$z^{1}$, $z^{2}$...., $z^{m}$}
- Sample a mini batch of **m** training examples {$x^{1}$, $x^{2}$...., $x^{m}$}
- Update **D** by doing one gradient descent step on its loss function: <br/>
   $$J_{D}= -\frac{1}{m} \sum_{m}^{i=1} \left [ log D(x^{(i)}) +   log(1- D(G(z^{(i)})))\right ]$$
- Update **G** by doing one gradient descent step on its loss function: <br/>
   $$J_{G}= -\frac{1}{m} \sum_{m}^{i=1} \left [ log D(G(z^{(i)}))\right ]$$ 


**Training Math** <br>

<div><img src="img/GAN_probs.png" width =30%></div>

Based upon our Loss function
$$L(G, D) = \int_x \bigg( p_{r}(x) \log(D(x)) + p_g (x) \log(1 - D(x)) \bigg) dx$$
we can derive following statements:
- $\mathbb{E}_{x \sim p_{r}(x)} [\log D(x)]$ has no impact on the Generator $G$ during gradient descent

Without proof the optimal value for $D$ based on the necessity $\frac{d f(D(x))}{d D(x)} = 0$ of $f(x) = p_r(x)\log(D(x))+p_g(x)\log(1−D(x))$
$$D^*(x) = \frac{p_{r}(x)}{p_{r}(x) + p_g(x)} \in [0, 1]$$ 
Once the generator is trained to its optimal, $p_g$ gets very close to $p_r$. When $p_g = p_r$, $D∗(x)$ becomes $1/2$.

For the global optimum when $G$ and $D$ take the optimal value 

$$\begin{aligned}
L(G, D^*) 
&= \int_x \bigg( p_{r}(x) \log(D^*(x)) + p_g (x) \log(1 - D^*(x)) \bigg) dx \\
&= \log \frac{1}{2} \int_x p_{r}(x) dx + \log \frac{1}{2} \int_x p_g(x) dx \\
&= -2\log2
\end{aligned}$$

**Representation of Loss Function $L$**

What we want to show with this proof is how the Loss function of the GAN Network is related to the Jason-Shannon divergence.

$$\begin{aligned}
D_{JS}(p_{r} \| p_g) 
=& \frac{1}{2} D_{KL}(p_{r} || \frac{p_{r} + p_g}{2}) + \frac{1}{2} D_{KL}(p_{g} || \frac{p_{r} + p_g}{2}) \\
=& \frac{1}{2} \bigg( \log2 + \int_x p_{r}(x) \log \frac{p_{r}(x)}{p_{r} + p_g(x)} dx \bigg) + \\& \frac{1}{2} \bigg( \log2 + \int_x p_g(x) \log \frac{p_g(x)}{p_{r} + p_g(x)} dx \bigg) \\
=& \frac{1}{2} \bigg( \log4 + L(G, D^*) \bigg)
\end{aligned}$$

thus $ L(G, D^*) = 2D_{JS}(p_{r} \| p_g) - 2\log 2$ and following that the best Generator $G^*$, meaing JS-divergence becomes $0$ and with that that $p_g = p_r$ will result in $L(G^*, D^*) = -2\log2$


**Important Discoveries**

- Batch normalization is a must in both networks.
- Fully hidden connected layers are not a good idea.
- Avoid pooling, simply stride your convolutions!
- ReLU activations are your friend (almost always).
- Vanilla GANs could work on simple datasets, but DCGANs are far better.
- DCGANS are solid baseline to compare with your fancy new state-of-the-art GAN algorithm.

# Deep Convolutional Generative Neural Networks
<div><img src="img/DCGAN.png" width=50%></div>

**Deconvolution Network cuppled with a CNN**

# Problems with GANS

**Nash Equilibrium** 

Two models are trained simultaneously to find a Nash equilibrium to a two-player non-cooperative game. However, each model updates its cost independently with no respect to another player in the game.

Suppose one player takes control of $x$ to minimize $f_1(x)=xy$, while at the same time the other player constantly updates $y$ to minimize $f_2(y)=−xy$.
Because $\frac{∂f_1}{∂x}=y$ and $\frac{∂f_2}{∂y}=−x$, we update $x$ with $x−η⋅y$ and $y$ with $y+η⋅x$ simulitanously in one iteration, where $η$ is the learning rate. Once $x$ and $y$ have different signs, every following gradient update causes huge oscillation and the instability gets worse in time.
<div><img src="img/nash_equilibrium.png" width=40%></div>

**Low Dimensional Support**

This problem is based on the assumption that every realworld data set $p_r$ only appears to have artifically high dimensions. This means it can be represented in a lower dimenional manifold. In simple terms your input data for example being vectors $x \in \mathbf{R}^n$ in n-dimensional space can be represented with a lower dimensionality by the latent space $Z$ by $z \in \mathbf{R}^m$ with the strict condition $m < n$. Because the generator is pretty much upscaling a random input of lower dimension to $p_g$ and the input data set $p_r$ lies in lower dimension itself there will always be a discriminator $D$ that will perfectly seperate real and fake samples becasue $D$ operates on the input dimensions $n$. Every added dimension gives the discriminator better "options" (more space) to be able to draw the boarder.
<br> I highly recommend checking out the full post for this <a href="https://lilianweng.github.io/lil-log/2017/08/20/from-GAN-to-WGAN.html#low-dimensional-supports"> here </a>. <br> 

**Vanishing Gradient**

Suppose our discriminator $D$ is perfect then $D(x) = 1, \enspace \forall x \in p_r$ and $D(x) = 0,  \enspace \forall x \in p_g$. Then the Loss function $L$ will become $0$ and so do the gradients. <br>
As a result, training a GAN faces a dilemma:

- If the discriminator behaves badly, the generator does not have accurate feedback and the loss function cannot represent the reality.
- If the discriminator does a great job, the gradient of the loss function drops down to close to zero and the learning becomes super slow or even jammed.
This dilemma clearly is capable to make the GAN training very tough.

**Mode Collapse** 

This is a problem concerning the generator network. Suppose the generator collapses to a setting where the output is fooling the discriminator most of the times but is in a low variety space (presenting the same generator output with only slight changes all the time). Mode collapse describes the problem that the model is only reproducing images or outputs based on a single class/label in the inputs. 

Next Tasks:
- Kernel Mapping, Manifold Learning, Multi-dimensional Scaling (MDS)